## Imports & helper functions

In [ ]:
# %pip install --upgrade openai

In [ ]:
import json


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


def parse_json(json_string):
  json_dict = json.loads(json_string)
  return json_dict


def dict_to_pretty_json(dictionary):
  pretty_json = json.dumps(dictionary, indent=2)
  return pretty_json


def write_dict_to_json(dictionary, file_path):
  with open(file_path, 'w') as json_file:
    json.dump(dictionary, json_file, indent=2)


In [ ]:
import configparser

def read_ini_file(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    ini_dict = {section: dict(config.items(section))
                for section in config.sections()}
    return ini_dict


In [ ]:
from openai import OpenAI

In [ ]:
import re


def remove_java_comments(java_source):
    # Regular expression to match Java comments (both single-line and multi-line)
    pattern = r"(//.*?$)|(/\*.*?\*/)"

    # Remove comments using the regular expression
    java_source_without_comments = re.sub(
        pattern, "", java_source, flags=re.MULTILINE | re.DOTALL)

    return java_source_without_comments.strip()


In [ ]:
def sentence(s):
  '''
  Capitalize the first letter of a string `s` and ensures that the string 
  ends with a period (if it's not already a sentence-ending punctuation).
  '''
  t = s.strip()
  if t[-1] in '.?!…~–—':
    return f'{t[0].upper()}{t[1:]}'
  else:
    return f'{t[0].upper()}{t[1:]}.'

In [ ]:
def transform_graph(graph):
	nodes = { node['data']['id']: node['data'] for node in graph['elements']['nodes'] }
	edges = {}
	for edge in graph['elements']['edges']:
		if 'label' in edge['data']:
			label = edge['data']['label']
		else:
			label = ','.join(edge['data']['labels'])
			edge['data']['label'] = label
		
		if label not in edges:
			edges[label] = []
		edges[label].append(edge['data'])
	return (nodes, edges)
	

In [ ]:
def invert(edgeList):
    prefix = "inv_"
    invertedEdges = []
    for edge in edgeList:
        invertedEdge = {
            'source': edge['target'],
            'target': edge['source'],
            'label': prefix + edge.get('label', ''),
            **{key: value for key, value in edge.items() if key not in ['source', 'target', 'label']}
        }
        invertedEdges.append(invertedEdge)
    return invertedEdges

def find_paths(edgeList1, edgeList2):
    source_mapping = {}
    for edge in edgeList1:
        source_mapping[edge['target']] = edge['source']

    paths = set()
    for edge in edgeList2:
        if edge['source'] in source_mapping:
            source1 = source_mapping[edge['source']]
            path = [source1, edge['source'], edge['target']]
            paths.add(tuple(path))

    return paths

## Parameters

In [ ]:
# If True: do not call the API, just print the prompts
only_print_prompt = False

In [ ]:
secrets = read_ini_file('config.ini')
project_name = secrets['project']['name']
project_desc = secrets['project']['desc']
ifile = secrets['project']['ifile']
(project_name,project_desc,ifile)

## Read graph file

To access knowledge graph extracted using javapers.

In [ ]:
graph = read_json_file(ifile)
nodes,edges = transform_graph(graph)

In [ ]:
nodes

## Connect to openai

In [ ]:
cliet_args = dict()

if 'apikey' in secrets['openai']:
  cliet_args['api_key'] = secrets['openai']['apikey']
if 'apibase' in secrets['openai']:
  cliet_args['base_url'] = secrets['openai']['apibase']
if 'model' in secrets['openai']:
  model = secrets['openai']['model']
else:
  model = "gpt-3.5-turbo"

(list(cliet_args.keys()), model)

In [ ]:
client = OpenAI(**cliet_args)
client.base_url

In [ ]:
# test the LLM server---create a completion
completion = client.chat.completions.create(
  model=model,
  messages=[{"role":"user","content":"What is your name?"}],
  temperature=0
)
# print the completion
print(completion.choices[0].message.content)

## Elements to be inspected

In [ ]:
methods = sorted([(pkg_id,cls_id,met_id) for pkg_id,cls_id,met_id in find_paths(edges['contains'], edges['hasScript']) if nodes[met_id]['properties']['visibility']=='public'])
len(methods)

In [ ]:
classes = sorted({(pkg,clz) for pkg,clz,_ in methods})
len(classes)

In [ ]:
packages = sorted({pkg for pkg,_ in classes})
len(packages),packages

In [ ]:
results = {pkg_id:{
	'qualifiedName': nodes[pkg_id]['properties']['qualifiedName'],
	'classes': {cls_id: {
		'qualifiedName': nodes[cls_id]['properties']['qualifiedName'],
		'methods': {met_id: {
			'qualifiedName': nodes[met_id]['properties']['qualifiedName']
		} for _,c,met_id in methods if c == cls_id}
	} for p,cls_id in classes if p == pkg_id}
} for pkg_id in packages}

results

## Ask LLM to classify methods into layers

In [ ]:
prompt1_template = '''Here are the layers in a layered software architecture and their responsibilities:

1. **Presentation Layer**: Manages the user interface, including defining UI elements and their behavior, displays information, reacts to user input, and updates UI views accordingly.
  
2. **Service Layer**: Orchestrates domain operations, encapsulates business logic, selects the appropriate business logic for a user request, and coordinates responses between the presentation and domain layers.

3. **Domain Layer**: Organizes and implements business logic, represents domain data and its behavior, and carries out the necessary computation for responding to user requests.

4. **Data Source Layer**: Communicates with databases, messaging systems, or other sources of data, performs CRUD operations, handles data conversion, and ensures data integrity before committing changes to the data source.

**Project Context**: {project_desc}

Consider the following Java method from the project:

```java
{method_src}
```

**Task**:

1. **Summarize the Method**: Briefly describe the primary responsibility of this method.
2. **Layer Comparison**: Compare the method's functionality to each of the layers described above. Determine which layer it best fits into and explain why. Consider the context of the project when determining the layer placement.
3. **Multiple Layers Assignment**: If you assigned the method to more than one layer, evaluate each assigned layer. Argue with yourself about which one of these layers fits the method best. Provide clear reasoning for your final decision.
4. **Single Layer Assignment**: If you assigned the method to only one layer, justify your choice. Convince me why this is the correct layer by providing specific examples and reasoning.  
5. **No Layer Assignment**: If you did not assign the method to any layer, reconsider your decision. Reflect on whether the method truly lies outside the defined layers (in such case, answer with "Cross-cutting) or if it leans towards one of the layers. Explain your reasoning.'''

prompt2 = "In conclusion, state the single layer that you think fits this method the most. Just answer with the name of the layer and nothing else."

In [ ]:
import time

timestr = time.strftime("%Y%m%d-%H%M%S")
timestr

In [ ]:
# timestr = '20240522-120247'

In [ ]:
with open(f'layerinator-{timestr}.log', 'a') as file:

    kind = 'class'
    current_pkg = None
    current_cls = None
    for pkg_id,cls_id,met_id in methods[:10]:
            
        if current_pkg != pkg_id:
            if current_pkg:
                file.write('\n\n===============================================\n\n')
                
            last_pkg = current_pkg
            current_pkg = pkg_id
            file.write('# ' + current_pkg + "\n")
        
        if current_cls != cls_id:
            file.flush()
            last_cls = current_cls
            current_cls = cls_id
            file.write('\t* ' + current_cls + "\n")

        if not 'layer' in results[pkg_id]['classes'][cls_id]['methods'][met_id] \
                or results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer'] == 'undefined':

            file.write('\t\t- ' + met_id + "\n")

            package = nodes[pkg_id]
            clasz = nodes[cls_id]
            method = nodes[met_id]

            method_name = method['properties']['simpleName']
            
            prompt1 = prompt1_template.format(
                project_desc=project_desc,
                method_src=method['properties']["sourceText"]
            )
            if only_print_prompt:
                file.write('\t\t\t' + prompt1)
                file.write("\n\n")
            else:
                response = None
                try:
                    response = client.chat.completions.create(
                        model=model,
                        messages=[{
                            "role": "user",
                            "content": prompt1
                        }],
                        temperature=0)
                    ast_message = response.choices[0].message
                    
                    file.write('\t\t\t' + "[USER]\n\n")
                    file.write('\t\t\t' + prompt1)
                    file.write("\n\n")
                    file.write('\t\t\t' + "[LLM]\n\n")
                    file.write('\t\t\t' + ast_message.content)
                    file.write("\n\n")

                    response = client.chat.completions.create(
                        model=model,
                        messages=[{
                            "role": "user",
                            "content": prompt1
                        }, 
                        ast_message,
                        {
                            "role": "user",
                            "content": prompt2
                        }],
                        temperature=0)
                    answer = response.choices[0].message.content
                    
                    file.write('\t\t\t' + "[USER]\n\n")
                    file.write('\t\t\t' + prompt2)
                    file.write("\n\n")
                    file.write('\t\t\t' + "[LLM]\n\n")
                    file.write('\t\t\t' + answer)
                    file.write("\n\n")

                    results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer'] = answer
                except:
                    answer = None
                    file.write('\t\t\t' + (str(response) if response else "no response"))
                    file.write("\n\n")
                    results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer'] = "undefined"
                    
            file.write("\n\n")

    file.write("ALL RESULTS:\n\n")
    file.write(dict_to_pretty_json(results))


In [ ]:
if not only_print_prompt:
  write_dict_to_json(results, f"{project_name}-layers-{timestr}.json")

In [ ]:
# results = read_json_file(f"{project_name}-layers-20240522-120247.json")
# results

In [ ]:
def count_layer_occurrences(input_dict):
    layer_count = {}

    for _, details in input_dict.items():
        layer = details.get("layer")
        if layer:
            if layer in layer_count:
                layer_count[layer] += 1
            else:
                layer_count[layer] = 1

    return layer_count

In [ ]:
results

In [ ]:
for pkg_id in results:
	for cls_id in results[pkg_id]['classes']:
		results[pkg_id]['classes'][cls_id]['layers'] = count_layer_occurrences(results[pkg_id]['classes'][cls_id]['methods'])

results

In [ ]:
def sum_layer_counts(input_dicts):
    layer_count = {}

    for _, details in input_dicts.items():
        layers = details.get("layers", {})
        for layer, count in layers.items():
            if layer in layer_count:
                layer_count[layer] += count
            else:
                layer_count[layer] = count

    return layer_count

In [ ]:
for pkg_id in results:
	results[pkg_id]['layers'] = sum_layer_counts(results[pkg_id]['classes'])

results

In [ ]:
write_dict_to_json(results, f"{project_name}-layers-recap-{timestr}.json")

In [ ]:
rows = []

for pkg_id in results:
	for cls_id in [c for c in results[pkg_id]['classes']]:
		for met_id in [m for m in results[pkg_id]['classes'][cls_id]['methods']]:
			rows.append((pkg_id,cls_id,met_id,results[pkg_id]['classes'][cls_id]['methods'][met_id]['layer']))

rows

In [ ]:
header = ("package", "class", "method", "layer")

In [ ]:
import csv

with open(f"{project_name}-layers1-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [ ]:
rows = []

for pkg_id in results:
	for cls_id in [c for c in results[pkg_id]['classes']]:
		for layer in results[pkg_id]['classes'][cls_id]['layers']:
			rows.append((pkg_id,cls_id,layer,results[pkg_id]['classes'][cls_id]['layers'][layer]))

rows

In [ ]:
header = ("package", "class", "layer", "count")

In [ ]:
import csv

with open(f"{project_name}-layers2-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [ ]:
rows = []

for pkg_id in results:
	for cls_id in [c for c in results[pkg_id]['classes']]:
		for layer in results[pkg_id]['classes'][cls_id]['layers']:
			rows.append((pkg_id,cls_id,layer,results[pkg_id]['classes'][cls_id]['layers'][layer]/sum(results[pkg_id]['classes'][cls_id]['layers'].values())))

rows

In [ ]:
import csv

with open(f"{project_name}-layers3-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)